In [1]:
import numpy as np
import pandas as pd
import pypokerengine as p
from pypokerengine.api.emulator import Emulator

In [2]:
emulator = Emulator()

In [3]:
players_info = {
    'uuid-0': {'name': 'player0', 'stack': 1000},
    'uuid-1': {'name': 'player1', 'stack': 1000},
    'uuid-2': {'name': 'player2', 'stack': 1000},
    'uuid-3': {'name': 'player3', 'stack': 1000},
    'uuid-4': {'name': 'player4', 'stack': 1000},
    'uuid-5': {'name': 'player5', 'stack': 1000}
}

emulator.set_game_rule(player_num=len(players_info),
                       max_round=5,
                       small_blind_amount=10,
                       ante_amount=0)

In [4]:
initial_state = emulator.generate_initial_game_state(players_info)
initial_state

{'round_count': 0,
 'small_blind_amount': 10,
 'street': 0,
 'next_player': None,
 'table': <pypokerengine.engine.table.Table at 0x2c4fc805c18>}

In [5]:
i = 0
event = [{'type': ''}]
events = {}

while event[-1]['type'] != 'event_game_finish':
    j = 0
    events[i] = {}
    
    if i == 0:
        game_state, event = emulator.start_new_round(initial_state)
    else:
        game_state, event = emulator.start_new_round(game_state)
    
    events[i][j] = event
    
    while event[-1]['type'] != 'event_round_finish' \
    and event[-1]['type'] != 'event_game_finish':
        j += 1
        
        action = np.random.choice(a=event[-1]['valid_actions'],
                                  size=1,
                                  p=[0.7 if e['action'] == 'call' else 0.3 / (len(event[-1]['valid_actions']) - 1) for e in event[-1]['valid_actions']])[0]['action']
        
        for va in event[-1]['valid_actions']:
            if va['action'] == action and action != 'raise':
                bet_amount = va['amount']
            elif va['action'] == action and action == 'raise':
                flip = np.random.choice(a=['min', 'max'],
                                        size=1,
                                        p=[1/2, 1/2])[0]
                bet_amount = va['amount'][flip]
            else:
                pass
            
        game_state, event = emulator.apply_action(game_state=game_state,
                                                  action=action,
                                                  bet_amount=bet_amount)
        events[i][j] = event
    
    if event[-1]['type'] == 'event_round_finish':
        for player in game_state['table'].seats.players:
            if player.stack == 0:
                player.append_chip(1000)
    
    i += 1

In [6]:
events

{0: {0: [{'type': 'event_new_street',
    'street': 'preflop',
    'round_state': {'street': 'preflop',
     'pot': {'main': {'amount': 30}, 'side': []},
     'community_card': [],
     'dealer_btn': 0,
     'next_player': 3,
     'small_blind_pos': 1,
     'big_blind_pos': 2,
     'round_count': 1,
     'small_blind_amount': 10,
     'seats': [{'name': 'player0',
       'uuid': 'uuid-0',
       'stack': 1000,
       'state': 'participating'},
      {'name': 'player1',
       'uuid': 'uuid-1',
       'stack': 990,
       'state': 'participating'},
      {'name': 'player2',
       'uuid': 'uuid-2',
       'stack': 980,
       'state': 'participating'},
      {'name': 'player3',
       'uuid': 'uuid-3',
       'stack': 1000,
       'state': 'participating'},
      {'name': 'player4',
       'uuid': 'uuid-4',
       'stack': 1000,
       'state': 'participating'},
      {'name': 'player5',
       'uuid': 'uuid-5',
       'stack': 1000,
       'state': 'participating'}],
     'action_histo

In [7]:
pd.DataFrame.from_dict(data=events[0][0][0]['round_state'], orient='index').reset_index()

,index,0
0,street,preflop
1,pot,"{'main': {'amount': 30}, 'side': []}"
2,community_card,[]
3,dealer_btn,0
4,next_player,3
5,small_blind_pos,1
6,big_blind_pos,2
7,round_count,1
8,small_blind_amount,10
9,seats,"[{'name': 'player0', 'uuid': 'uuid-0', 'stack'..."


In [8]:
pd.DataFrame.from_dict(data=events[1][0][0]['round_state'], orient='index').reset_index()

,index,0
0,street,preflop
1,pot,"{'main': {'amount': 30}, 'side': []}"
2,community_card,[]
3,dealer_btn,1
4,next_player,4
5,small_blind_pos,2
6,big_blind_pos,3
7,round_count,2
8,small_blind_amount,10
9,seats,"[{'name': 'player0', 'uuid': 'uuid-0', 'stack'..."
